# <center>Challenge Etermax | Cecilia Trejo </center>

#### Objetivo 
Determinar los tópicos predominantes en cada conversación del corpus adjunto. 

#### Notas:
- Se adjunta corpus (en formato csv) compuesto por la desgrabación de 640 llamadas.
- Cada desgrabación corresponde a un llamado en el que interviene un cliente y un agente de un contact center.

#### Lineamientos:

- El lenguaje a utilizar es Python, y se debe entregar una Notebook en donde se puedan visualizar todos los pasos ejecutados, desde la carga del corpus, hasta la predicción de cada conversación realizada por el modelo.
- Se pueden utilizar las librerías y técnicas que se consideren, no hay limitación en cuanto a este punto.
- El código deberá ser compartido a través de una cuenta de Git.


#### Instalación de Librerías

In [1]:
!pip install spacy
!pip install pandas 
!pip install nltk 
!pip install gensim 
!pip install pyLDAvis

#### Importación de Librerías 
En esta sección, se importan todas las librerías necesarias para realizar el procesamiento y modelado de tópicos. Estas bibliotecas incluyen Gensim para el modelado de tópicos, spaCy para el procesamiento de texto, nltk para la gestión de stopwords y pyLDAvis para la visualización de resultados. 

In [66]:
# Gensim
import gensim
from gensim import corpora, models, similarities
from gensim.utils import simple_preprocess
from gensim.models.coherencemodel import CoherenceModel

# Manipulación de Archivos
import pandas as pd
import os

# Spacy
import spacy

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Visualización LDA
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis



In [67]:
# Se descargan los datos necesarios para el tokenizador de NLTK.
nltk.download('punkt') 

# Se descarga las stopwords en español y se almacenan en la variable stop_words.
nltk.download('stopwords')  

# Se descarga el lematizador de NLTK y se almacena en la variable lemmatizer
nltk.download('wordnet')

# Se descarga el modelo en español de Spacy
!python -m spacy download es_core_news_sm


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\keepc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\keepc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\keepc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------- 0.1/12.9 MB 819.2 kB/s eta 0:00:16
     --------------------------------------- 0.2/12.9 MB 893.0 kB/s eta 0:00:15
      --------------------------------------- 0.3/12.9 MB 1.1 MB/s eta 0:00:12
     - -------------------------------------- 0.5/12.9 MB 1.7 MB/s eta 0:00:08
     -- ------------------------------------- 0.7/12.9 MB 2.1 MB/s eta 0:00:06
     -- ------------------------------------- 0.9/12.9 MB 2.3 MB/s eta 0:00:06
     --- ------------------------------------ 1.2/12.9 MB 2.6 MB/s eta 0:00:05
     --- ------------------------------------ 1.3/12.9 MB 2.6 MB/s eta 0:00:05
     ---- ----------------------------------- 1.4/12.9 MB 2.7 MB/s eta 0:00:05
     ---- ----------------------------------- 1.6/12.9 MB 2.6 MB/s eta 

#### Versiones de librerías

In [68]:
print("Versión de gensim:", gensim.__version__) 
print("Versión de Spacy:", spacy.__version__) 
print("Versión de NLTK:", nltk.__version__) 
print("Versión de Pandas:", pd.__version__) 
print("Versión de pyLDAvis:", pyLDAvis.__version__)

Versión de gensim: 4.3.2
Versión de Spacy: 3.7.1
Versión de NLTK: 3.8.1
Versión de Pandas: 2.1.1
Versión de pyLDAvis: 3.4.0


#### Carga del DataFrame:
Establece la variable folder como la ubicación de los archivos de datos alojados en la carpeta "./datasets" y realiza una lista de los archivos que contiene la carpeta folder.

In [69]:
folder = "./datasets"

# Se listan los archivos del path folder
os.listdir(folder)

['34701-10-07-17.csv',
 '34701-12-31-48.csv',
 '34701-12-57-22.csv',
 '34701-13-27-47.csv',
 '34701-13-56-30.csv',
 '34701-14-03-34.csv',
 '34701-14-38-15.csv',
 '34701-14-49-56.csv',
 '34701-15-40-27.csv',
 '34701-16-34-33.csv',
 '34701-18-35-24.csv',
 '34701-18-53-05.csv',
 '34701-20-19-55.csv',
 '34705-09-20-44.csv',
 '34705-10-47-39.csv',
 '34705-11-02-13.csv',
 '34705-12-32-07.csv',
 '34705-13-04-55.csv',
 '34705-13-14-18.csv',
 '34705-13-24-44.csv',
 '34705-13-34-12.csv',
 '34705-13-43-17.csv',
 '34705-13-45-12.csv',
 '34705-13-54-09.csv',
 '34705-14-39-45.csv',
 '34705-14-48-49.csv',
 '34705-14-58-31.csv',
 '34705-16-33-41.csv',
 '34705-16-39-20.csv',
 '34705-17-01-54.csv',
 '34705-17-06-32.csv',
 '34705-17-07-55.csv',
 '34705-17-16-06.csv',
 '34705-17-16-55.csv',
 '34705-17-34-34.csv',
 '34705-18-08-15.csv',
 '34705-19-22-00.csv',
 '34705-19-23-43.csv',
 '34705-19-32-20.csv',
 '34705-19-55-48.csv',
 '34705-20-04-19.csv',
 '34705-20-13-34.csv',
 '34705-20-17-47.csv',
 '34705-21-

In [70]:
# Se crea un DataFrame vacío en la variable 'df' utilizando la librería Pandas.
df = pd.DataFrame()

# Se crea un ciclo for que itera a través de los archivos de la folder 
for file in os.listdir(folder):
    print("File = ", file)
    
    # Se concatena folder con el file en cuestión para construir la ruta completa del archivo
    file_path = os.path.join(folder, file)
    print("File path = ", file_path)
    
    # Se lee el archivo CSV en un DataFrame. Se utiliza como parámetro de delimitación la pleca (|) para que las interacciones se alineen en una sola columna.
    file_df = pd.read_csv(file_path, sep="|")
    
    # Se agrega una columna al DataFrame que indica de qué archivo es esta data
    file_df["file_path"] = file_path
    
    # Se concatena file_df al DataFrame principal que antes estaba vacío
    df = pd.concat([df, file_df], ignore_index=True)

File =  34701-10-07-17.csv
File path =  ./datasets\34701-10-07-17.csv
File =  34701-12-31-48.csv
File path =  ./datasets\34701-12-31-48.csv
File =  34701-12-57-22.csv
File path =  ./datasets\34701-12-57-22.csv
File =  34701-13-27-47.csv
File path =  ./datasets\34701-13-27-47.csv
File =  34701-13-56-30.csv
File path =  ./datasets\34701-13-56-30.csv
File =  34701-14-03-34.csv
File path =  ./datasets\34701-14-03-34.csv
File =  34701-14-38-15.csv
File path =  ./datasets\34701-14-38-15.csv
File =  34701-14-49-56.csv
File path =  ./datasets\34701-14-49-56.csv
File =  34701-15-40-27.csv
File path =  ./datasets\34701-15-40-27.csv
File =  34701-16-34-33.csv
File path =  ./datasets\34701-16-34-33.csv
File =  34701-18-35-24.csv
File path =  ./datasets\34701-18-35-24.csv
File =  34701-18-53-05.csv
File path =  ./datasets\34701-18-53-05.csv
File =  34701-20-19-55.csv
File path =  ./datasets\34701-20-19-55.csv
File =  34705-09-20-44.csv
File path =  ./datasets\34705-09-20-44.csv
File =  34705-10-47-

#### DataFrame

In [71]:
# En este print se puede observar que el DataFrame ya no está vacio y que aparece la nueva columna file_path que cree en la celda anterior
df

,Pregunta,file_path
0,"Hola , mi nombre es Valeria . Disculpe , no e...",./datasets\34701-10-07-17.csv
1,Buenas tardes . punto Mi nombre es José Gutié...,./datasets\34701-12-31-48.csv
2,Hola . Buenas tardes . Habla con María Azcára...,./datasets\34701-12-57-22.csv
3,"Buenas tardes , Francisco Moreno . , que mire...",./datasets\34701-13-27-47.csv
4,"al final . Hola , buenas tardes con Karen . M...",./datasets\34701-13-56-30.csv
...,...,...
635,barba . Buenas tardes . . tengo una consulta ...,./datasets\34809-12-21-48.csv
636,Hola . Buenos días . Mi nombre es Paola . Le ...,./datasets\34809-12-37-51.csv
637,Buenas tardes . Bienvenido . . mi nombre es E...,./datasets\34809-12-42-09.csv
638,". Mira , tengo una pura . Es una compra onlin...",./datasets\34809-12-45-28.csv


#### Preprocesamiento del texto: 
Esto incluye tokenización, eliminación de stopwords y lematización. 
Se utiliza NLTK y Spacy con el modelo de lenguaje en español ("es_core_news_sm") para realizar la lematización de las conversaciones.

In [72]:
# Se convierten las palabras a minúsculas y luego se tokeniza
def lower_and_tokenize(text):
    return word_tokenize(text.lower())
df['Pregunta'] = df['Pregunta'].apply(lower_and_tokenize)

stop_words = set(stopwords.words('spanish'))  

# Se analiza palabra por palabra del DataFrame y se filtran aquellas que no son stopwords
def remove_stopwords(text):
    return [word for word in text if word not in stop_words]
df['Pregunta'] = df['Pregunta'].apply(remove_stopwords)

# Se realiza un preprocesamiento utilizando la función simple_preprocess de Gensim para la eliminación de acentos y caracteres no alfabéticos.
def preprocess(text):
    return [word for word in simple_preprocess(' '.join(text), deacc=True)]
df['Pregunta'] = df['Pregunta'].apply(preprocess)

In [73]:
lemmatizer = WordNetLemmatizer()

def lemmatize_words(word_list):
    return [lemmatizer.lemmatize(word) for word in word_list]

# Se lematiza cada lista de palabras en 'Pregunta'
df['Pregunta'] = df['Pregunta'].apply(lemmatize_words)

# Se une las listas de palabras de 'Pregunta' en strings
df['Pregunta'] = df['Pregunta'].apply(' '.join)


In [74]:
df['Pregunta']

0      hola nombre valeria disculpe escuche nombre jo...
1      buenas tardes punto nombre jose gutierrez gust...
2      hola buenas tardes habla maria azcarate consul...
3      buenas tardes francisco moreno mire pedidos mi...
4      final hola buenas tardes karen mira quiero hac...
                             ...                        
635    barba buenas tardes consulta pedido iba llegar...
636    hola buenos dia nombre paola quiero hacer cons...
637    buenas tardes bienvenido nombre eduardo quien ...
638    mira pura compra online semana atras producto ...
639    buenas tardes bienvenida nombre eduardo gusto ...
Name: Pregunta, Length: 640, dtype: object

In [75]:
# Se aplica el lematizador de Spacy para conservar solo las palabras con categorías gramaticales especificadas (sustantivos, adjetivos y verbos) y se devuelve el texto de la conversación con las palabras lematizadas.

def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB"]):
    nlp = spacy.load("es_core_news_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return texts_out


lemmatized_texts = lemmatization(df['Pregunta'])

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return final

list_data_words = gen_words(lemmatized_texts)

In [76]:
# Acá ya se ve que las conversaciones de la columna 'Pregunta" no tienen acentos ni stopwords
df.head()

,Pregunta,file_path
0,hola nombre valeria disculpe escuche nombre jo...,./datasets\34701-10-07-17.csv
1,buenas tardes punto nombre jose gutierrez gust...,./datasets\34701-12-31-48.csv
2,hola buenas tardes habla maria azcarate consul...,./datasets\34701-12-57-22.csv
3,buenas tardes francisco moreno mire pedidos mi...,./datasets\34701-13-27-47.csv
4,final hola buenas tardes karen mira quiero hac...,./datasets\34701-13-56-30.csv


#### Creación de Diccionario y Corpus
En esta sección, se crea un diccionario de palabras (id2word) a partir de la lista de palabras tokenizadas (list_data_words). Luego, se crea un corpus en forma de representación de  bolsa de palabras utilizando el diccionario. Esto es necesario para el modelado de tópicos. 

##### bag-of-words (BoW)
Al utilizar la técnica de bag-of-words (BoW) convertimos la conversación en su vector equivalente de números. 
Se crea una tupla (ID de palabra, frecuencia de aparición) para cada palabra. La variable new_tuple almacena estas tuplas.

In [77]:
# Se crea un diccionario de palabras a partir de la lista de palabras procesadas
id2word = corpora.Dictionary(list_data_words)

# Se crea un corpus de tuplas (bag-of-words) a partir del diccionario y las palabras procesadas
tuple_corpus = []
for word in list_data_words:
    new_tuple = id2word.doc2bow(word)
    tuple_corpus.append(new_tuple)
    
print(tuple_corpus)    

[[(0, 1), (1, 1), (2, 1), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 2), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 2), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 2), (55, 2), (56, 1), (57, 2), (58, 3), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 2), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 2), (84, 5), (85, 1), (86, 1)], [(15, 1), (19, 1), (22, 1), (23, 1), (26, 3), (29, 1), (34, 2), (36, 1), (41, 2), (49, 1), (50, 1), (54, 1), (57, 1), (58, 4), (67, 1), (77, 1), (78, 1), (85, 1), (87, 3), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1)

#### Entrenamiento de modelo con LDA (Latent Dirichlet Allocation)
En esta sección se entrena un modelo de tópicos utilizando el algoritmo LDA de Gensim. Se especifican los parámetros del modelo, como el número de tópicos, el tamaño de lote, el número de pases, etc.

In [78]:
# Se entrena un modelo LDA utilizando Gensim con los parámetros especificados.

lda_model = gensim.models.ldamodel.LdaModel(corpus=tuple_corpus, id2word=id2word, num_topics=7, random_state=100, update_every=1, chunksize=200, passes=10, alpha="auto")

In [79]:
# Se imprimen los tópicos del modelo LDA.

topics = lda_model.print_topics(num_topics=7, num_words=2)  
for topic in topics:
    print(topic)

(0, '0.021*"auto" + 0.018*"reserva"')
(1, '0.057*"producto" + 0.032*"cosa"')
(2, '0.051*"pedido" + 0.036*"hacer"')
(3, '0.022*"unilever" + 0.015*"vivir"')
(4, '0.029*"favor" + 0.027*"pedido"')
(5, '0.064*"unidad" + 0.060*"gramo"')
(6, '0.039*"hacer" + 0.037*"decir"')


#### Visualización de los datos obtenidos

In [80]:
# Visualización a través de la librería pyLDAvis
pyLDAvis.enable_notebook()
lda_data = gensimvis.prepare(lda_model, tuple_corpus, id2word, mds="mmds", R=30)

pyLDAvis.display(lda_data)

C:\Users\keepc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\manifold\_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


#### Tópicos descubiertos por el modelo LDA durante el proceso de entrenamiento
- La función **show_topics()** es específica de Gensim y proporciona información sobre los tópicos descubiertos durante el entrenamiento del modelo LDA
- Se muestran los tópicos junto con las palabras más relevantes que contribuyen a cada tópico y sus respectivas ponderaciones
- El número que representa al identificador de un tópico es único. Esta identificación es interna y se genera automáticamente por el modelo LDA durante el proceso de entrenamiento. No tiene relación directa con ningún tópico específico en el conjunto de datos original ni con ningún orden específico de tópicos

In [81]:
lda_model.show_topics()

[(0,
  '0.021*"auto" + 0.018*"reserva" + 0.016*"dia" + 0.016*"faltar" + 0.014*"hija" + 0.014*"champu" + 0.014*"mama" + 0.013*"veintiuno" + 0.013*"total" + 0.011*"lado"'),
 (1,
  '0.057*"producto" + 0.032*"cosa" + 0.021*"decir" + 0.020*"unidad" + 0.019*"venir" + 0.018*"pedido" + 0.013*"correcto" + 0.012*"cambiar" + 0.011*"hacer" + 0.011*"kilo"'),
 (2,
  '0.051*"pedido" + 0.036*"hacer" + 0.027*"llegar" + 0.024*"decir" + 0.023*"diar" + 0.014*"gracia" + 0.014*"llamar" + 0.012*"bueno" + 0.011*"perfecto" + 0.011*"tarde"'),
 (3,
  '0.022*"unilever" + 0.015*"vivir" + 0.011*"aplicar" + 0.011*"seguimiento" + 0.009*"mario" + 0.009*"preocupo" + 0.009*"pedir" + 0.008*"hermana" + 0.008*"sumar" + 0.008*"familia"'),
 (4,
  '0.029*"favor" + 0.027*"pedido" + 0.020*"numero" + 0.018*"indicar" + 0.016*"el" + 0.016*"momento" + 0.016*"punto" + 0.015*"caso" + 0.014*"producto" + 0.014*"correcto"'),
 (5,
  '0.064*"unidad" + 0.060*"gramo" + 0.018*"color" + 0.017*"personar" + 0.015*"kilograr" + 0.013*"hielo" + 0.

#### Tópicos descubiertos por el modelo LDA para cada conversación del corpus otorgado
- Después de ejecutar las siguientes funciones se imprimen los resultados mostrando por cada conversación, su bag-of-words correspondiente y los tópicos relevandos, primero ordenados por número de tópico y luego ordenados por probabilidad


In [82]:
# Se crea una función para asignar tópicos a una conversación específica
def assign_topics_to_conversation(text):
    
    # Dentro de la función, se crea una representación de Bolsa de Palabras (BoW) para la conversación text utilizando el diccionario id2word. La función simple_preprocess se utiliza para preprocesar el texto, convirtiéndolo en minúsculas y eliminando acentos y caracteres no alfabéticos.
    bow = id2word.doc2bow(simple_preprocess(text, deacc=True))
    
    print("CONVERSACION: ")
    print(text)
    print("Bag Of Words(BOW) DE LA CONVERSACION: ")
    print(bow)
    topic_distribution = lda_model.get_document_topics(bow)
    print("TOPIC_DISTRIBUTION: ")
    print(topic_distribution)
    
    # Se ordenan los tópicos por probabilidad y se obtiene el tópico más probable
    
    def sort_by_second_element(x):
        return x[1]

    topic_distribution.sort(key=sort_by_second_element, reverse=True)

    print("TOPIC_DISTRIBUTION ORDENADO x PROBABILIDAD: ")
    print(topic_distribution)
    print("_______________________________________")
    # Devolver el número de tópico más probable
    return topic_distribution[0][0]

# Se aplica esta función a la columna 'Pregunta' del DataFrame y se crea una nueva columna llamada 'Topic' en el DataFrame para almacenar los tópicos asignados.
df['Topic'] = df['Pregunta'].apply(assign_topics_to_conversation)
df

CONVERSACION: 
hola nombre valeria disculpe escuche nombre jose jose mira sabes bueno constantemente asi digo tiempo cuarentena ahora pedido recibir ahora hecho guardando fecha entrega lunes minuto llegaron sali comprar entonces rehaciendo dejaron conserjeria llego ahora pedido perdio cadena frio entonces ciertos productos dejando pasar todavia congelador carne pollo disfrutas llegaron olor dinero casa frutilla incomible tan tan plata igual molesta salmon salmon tampoco congelado entonces peligroso venian cajas mojadas aseguraron perdia digamos haberse entregado mismo dia preocupaba productos encadene frio parecer menos encaja caja carton bueno venia viene realidad frutillas vienen mal olor tia tia todavia abriendo caja vienen mala preocupa viene hablando ver caja seis cinco nueve mil nueve mil do final pedido se pasa creo abri refrigerados carne venia viajando venia liquido carne bien deletreo latina ene arroba gmail punto com entonces manda correo respondo perfecto ultima casa cosas 

TOPIC_DISTRIBUTION: 
[(0, 0.08375203), (1, 0.31779087), (2, 0.18985154), (4, 0.12256843), (6, 0.28518254)]
TOPIC_DISTRIBUTION ORDENADO x PROBABILIDAD: 
[(1, 0.31779087), (6, 0.28518254), (2, 0.18985154), (4, 0.12256843), (0, 0.08375203)]
_______________________________________
CONVERSACION: 
hola buenos tele usted conecta mire primera vez pedido online grupo mire siete credito dice ningun pedido entonces dice tarjeta credito ser tia chile pues quiero saber si si usted aparece incluso horario van cosas pedido dice pedido ahora saber master diez cuatro cuatro tres cuatro siete do hice do veces decia eh peso ayer mala experiencia lider peso telefono telefono computador llamado hagan cobro medido total total ingreso banco dijo ser receta ciudad tarjeta nacional entonces se si indique entonces voy tener hacer segundo punto puede ingresar prendiendo abriendo computador dejeme alo ingresando computador cerrar entonces punto posible entonces ingresar ingresar clave dice ingresaste incorrecta t

,Pregunta,file_path,Topic
0,hola nombre valeria disculpe escuche nombre jo...,./datasets\34701-10-07-17.csv,2
1,buenas tardes punto nombre jose gutierrez gust...,./datasets\34701-12-31-48.csv,4
2,hola buenas tardes habla maria azcarate consul...,./datasets\34701-12-57-22.csv,2
3,buenas tardes francisco moreno mire pedidos mi...,./datasets\34701-13-27-47.csv,2
4,final hola buenas tardes karen mira quiero hac...,./datasets\34701-13-56-30.csv,6
...,...,...,...
635,barba buenas tardes consulta pedido iba llegar...,./datasets\34809-12-21-48.csv,2
636,hola buenos dia nombre paola quiero hacer cons...,./datasets\34809-12-37-51.csv,6
637,buenas tardes bienvenido nombre eduardo quien ...,./datasets\34809-12-42-09.csv,2
638,mira pura compra online semana atras producto ...,./datasets\34809-12-45-28.csv,6


### DataFrame Final
La siguiente función crea una nueva columna en el DataFrame que figurará bajo el nombre de 'Topic Words'. 
- 'Topic Words': Contiene las palabras clave más importantes del tópico correspondiente a la conversación de esa fila
- 'Topic': Contiene el ID del tópico con la ponderación más alta para cada conversación

In [83]:
def show_topic_words(topic_id):
    topic_tuple_list = lda_model.show_topic(topic_id, topn=10)
    topic_words = [word for word, _ in topic_tuple_list]
    return " ".join(topic_words)

df['Topic Words'] = df['Topic'].apply(show_topic_words)
df


,Pregunta,file_path,Topic,Topic Words
0,hola nombre valeria disculpe escuche nombre jo...,./datasets\34701-10-07-17.csv,2,pedido hacer llegar decir diar gracia llamar b...
1,buenas tardes punto nombre jose gutierrez gust...,./datasets\34701-12-31-48.csv,4,favor pedido numero indicar el momento punto c...
2,hola buenas tardes habla maria azcarate consul...,./datasets\34701-12-57-22.csv,2,pedido hacer llegar decir diar gracia llamar b...
3,buenas tardes francisco moreno mire pedidos mi...,./datasets\34701-13-27-47.csv,2,pedido hacer llegar decir diar gracia llamar b...
4,final hola buenas tardes karen mira quiero hac...,./datasets\34701-13-56-30.csv,6,hacer decir tarjeta claro ver compra correo ve...
...,...,...,...,...
635,barba buenas tardes consulta pedido iba llegar...,./datasets\34809-12-21-48.csv,2,pedido hacer llegar decir diar gracia llamar b...
636,hola buenos dia nombre paola quiero hacer cons...,./datasets\34809-12-37-51.csv,6,hacer decir tarjeta claro ver compra correo ve...
637,buenas tardes bienvenido nombre eduardo quien ...,./datasets\34809-12-42-09.csv,2,pedido hacer llegar decir diar gracia llamar b...
638,mira pura compra online semana atras producto ...,./datasets\34809-12-45-28.csv,6,hacer decir tarjeta claro ver compra correo ve...
